# Hardware Testing Process — Rapport de laboratoire 

## Titre
Acquisition et analyse de signal

## Auteur
- Yildirim Arifcan

## Date
17-11-2025

## Objectifs
- Décrire le système expérimental et les instruments.
- Acquérir des signaux mesurables (tension, courant, capteur).
- Traiter et analyser les données (filtrage, transformée de Fourier, estimation de paramètres).
- Évaluer incertitudes et discuter les résultats.

## Matériel et instruments
- Oscilloscope: [DS1104]
- Générateur de fonctions: [DG1022]
- Câblage et adaptateurs

## Environnement
- Python


# Etape 1 : Détection automatique et gestion des instruments Rigol via PyVISA

Ce code a pour objectif d’automatiser la détection et l’utilisation des instruments RIGOL du laboratoire compatibles VISA, tels que le générateur de fonctions **DG1022** et l’oscilloscope **DS1104Z Plus**.
Il s’appuie sur trois éléments fondamentaux vus dans le syllabus :

* **Python** pour la programmation
* **VISA** (PyVISA + PyVISA-py) pour la communication matériel ↔ programme
* **SCPI** pour envoyer des commandes standardisées aux instruments

---

# 1) Classe `RigolInstrument`

Cette classe représente **un instrument Rigol individuel** (générateur, oscilloscope, etc.).

### ```python

class RigolInstrument:
def **init**(self, resource_name):
self.inst = pyvisa.ResourceManager('@py').open_resource(resource_name)
self.inst.timeout = 5000
self.inst.encoding = 'utf-8'
self.idn = self.get_idn()
self.instrument_type = self.classify_instrument()

````

### 🔍 Ce qui se passe ici :

1. **Ouverture de la ressource VISA**  
   `resource_name` ressemble à :  
   `USB0::0x1AB1::0x04CE::DS1ZC212301128::INSTR`  
   C’est l’identifiant unique du périphérique.

2. **Identification automatique de l’instrument**  
   Appel immédiat à :
   - `self.get_idn()` → récupère le `*IDN?`
   - `self.classify_instrument()` → détermine le type (oscillo/générateur)

---

# 2) Méthode `get_idn()`

```python
def get_idn(self):
    try:
        self.inst.write('*IDN?')
        time.sleep(0.1)
        return self.inst.read().strip()
    except pyvisa.errors.VisaIOError:
        return "Unknown"
````

### 🔍 Rôle :

* Envoie la commande SCPI standard `*IDN?`
* Lit la réponse, par ex. :
  `RIGOL TECHNOLOGIES,DS1104Z Plus,DS1ZC...,00.04.04.SP4`
* Si l’instrument ne répond pas → `"Unknown"`

Ce `*IDN?` est **le point de départ obligatoire** pour toute communication VISA (comme expliqué dans le syllabus).

---

# 3) Méthode `classify_instrument()`

```python
def classify_instrument(self):
    idn_upper = self.idn.upper()
    if "RIGOL" in idn_upper and "DG" in idn_upper:
        return "generator"
    elif "RIGOL" in idn_upper and ("DS" in idn_upper or "MSO" in idn_upper):
        return "oscilloscope"
    else:
        return "unknown"
```

### 🔍 Rôle :

Déterminer automatiquement le **type** d’instrument :

* `"DG"` → générateur DG1022
* `"DS"` ou `"MSO"` → oscilloscope (par ex. DS1104Z)

Cela permet de connecter automatiquement chaque appareil **sans hardcoder son adresse USB** → c’est exactement **l’exercice bonus** du syllabus (« Connect automatically one generator and one oscilloscope »).

---

# 4) Méthodes essentielles : `write`, `read`, `close`

```python
def write(self, command):
    self.inst.write(command)

def read(self):
    return self.inst.read()

def close(self):
    self.inst.close()
```

Ce sont des **wrappers** autour des fonctions PyVISA :

* `write(cmd)` : envoie une commande SCPI
* `read()` : lit une réponse
* `close()` : ferme proprement la connexion

Ces méthodes sont utilisées ensuite dans les fonctions du générateur et de l’oscilloscope.

---

# 5) Représentation textuelle

```python
def __str__(self):
    return f"{self.instrument_type.capitalize()} - IDN: {self.idn}"
```

Permet d’écrire :

```
Oscilloscope - IDN: RIGOL TECHNOLOGIES,DS1104Z Plus,...
```

---

# 6) Fonction `detect_rigol_instruments()`

```python
def detect_rigol_instruments():
    rm = pyvisa.ResourceManager('@py')
    devices = [d for d in rm.list_resources() if "USB" in d]

    generator = None
    oscilloscope = None
    print("Detecting Rigol instruments...")

    for dev in devices:
        rigol = RigolInstrument(dev)
        if rigol.instrument_type == "generator":
            generator = rigol
            print(f"Detected Generator: {generator}")
        elif rigol.instrument_type == "oscilloscope":
            oscilloscope = rigol
            print(f"Detected Oscilloscope: {oscilloscope}")

    return generator, oscilloscope
```

### 🔍 Rôle complet :

1. **Récupérer la liste des instruments VISA**
   Equivalent à :

   ```python
   rm.list_resources()
   ```

   Ce que le syllabus impose comme première étape.

2. **Filtrer les instruments USB**
   Les instruments du labo Rigol apparaissent au format `USB0::…`.

3. **Instancier la classe `RigolInstrument` pour chaque ressource**

4. **Classer automatiquement** :

   * Générateur DG1022
   * Oscilloscope DS1104Z
   * Autres

5. **Retourner un tuple** :

   ```python
   (generator, oscilloscope)
   ```

C’est exactement la solution demandée dans **le Challenge de la Session 1**.

---

# 🎯 Pourquoi ce code est utile pour le laboratoire ?

Il automatise plusieurs étapes :

### ✔ Identifier automatiquement les instruments

Plus besoin de copier manuellement l’adresse USB.

### ✔ Éviter les erreurs de connexion

Le code sait quel instrument est un oscilloscope et lequel est un générateur.

### ✔ Respecte complètement la logique VISA

* `ResourceManager`
* `open_resource()`
* `*IDN?`
* Chaînes SCPI

---

# Utilisation

> Nous avons développé une classe Python permettant de gérer automatiquement les instruments du laboratoire via PyVISA.
> Chaque instrument USB est détecté, interrogé via la commande SCPI `*IDN?`, puis classé selon son modèle (oscilloscope DS1104Z ou générateur DG1022).
> Cette structure permet d’abstraire la gestion matérielle et d’utiliser les instruments directement à travers des fonctions Python (write, read) tout en garantissant une communication fiable via le protocole VISA.

In [5]:

from find_instruments import detect_rigol_instruments
import time

generator, oscilloscope = detect_rigol_instruments()

# ==================================== FONCTIONS POUR LE GÉNÉRATEUR DE SIGNAL ====================================


def generator_timed_signal():
    """Active et désactive un signal toutes les secondes pendant 10 secondes."""
    for i in range(10):
        generator.write("OUTP1 ON")
        time.sleep(1)
        generator.write("OUTP1 OFF")
        time.sleep(1)



def generator_sine_signal():
    """Configure une onde sinusoïdale sur Channel 1 et une onde carrée sur Channel 2."""

    # Sine wave sur Channel 1 avec amplitude 5 V, fréquence 3000 Hz et offset 0 V
    generator.write("APPL:SIN 3000,5,0")
    time.sleep(1)  # Pause pour s’assurer que la commande est prise en compte
    generator.write("OUTP1 ON")
    time.sleep(5)  # Temps pour observer à l’oscillo
    generator.write("OUTP1 OFF")
    time.sleep(1)  # Pause pour s’assurer que la commande est prise en compte
    
    # Square wave sur Channel 2 avec amplitude 3 V, fréquence 1000 Hz et offset 0 V
    generator.write(" APPLy:SQUare:CH2 1000,3,0")
    time.sleep(1)  # Pause pour s’assurer que la commande est prise en compte
    generator.write("OUTP:CH2 ON")
    time.sleep(5)  # Temps pour observer à l’oscillo
    generator.write("OUTP:CH2 OFF")



def generator_exponential_signal():
    """Configure une onde exponentielle sur Channel 1."""

    # Exponential wave sur Channel 1 avec amplitude 5 V
    generator.write("APPL:USER EXP_RISE,1,5,0")
    time.sleep(1)
    generator.write("OUTP1 ON")
    time.sleep(5)  # Temps pour observer à l’oscillo
    generator.write("OUTP1 OFF")


# ==================================== FONCTIONS POUR L'OSCILLOSCOPE ====================================

def test_channel_1():
    """Active l'affichage du Channel 1 de l'oscilloscope."""
    # generator.write("OUTP1 ON")
    # time.sleep(1)
    oscilloscope.write("CHAN1:DISP ON")
    time.sleep(3)
    oscilloscope.write("CHAN1:DISP OFF")


# ============================================= MAIN PROGRAM ============================================

def observe_exponential_signal():
    """Observe le signal exponentiel fournit par le générateur sur l'oscilloscope."""
    generator.write("APPL:USER EXP_RISE,1,5,0")
    time.sleep(1)
    generator.write("OUTP1 ON")
    oscilloscope.write("CHAN1:DISP ON")
    oscilloscope.write("TIM:SCAL 0.002")  # Ajuste l'échelle de temps pour mieux voir le signal
    time.sleep(15)  # Temps pour observer à l’oscillo
    
    generator.write("OUTP1 OFF")
    oscilloscope.write("CHAN1:DISP OFF")
    

if __name__ == "__main__":
    if generator:
        print("Générateur de signal détecté.")
        #generator_timed_signal()
        #generator_sine_signal()
        #generator_exponential_signal()
        
    else:
        print("Générateur de signal non détecté !")


    if oscilloscope:
        print("Oscilloscope détecté.")
        test_channel_1()
        
    else:
        print("Oscilloscope non détecté !")

    if generator and oscilloscope:
        print("instruments détectés.")
        observe_exponential_signal()

        generator.close()
        oscilloscope.close()

USBError: [Errno 16] Resource busy

In [ ]:

## Traitement des données
- Importer données et métadonnées.
- Prétraitement: retrait de la composante continue, correction de gain, synchronisation.
- Filtrage: e.g. passe-bas Butterworth d'ordre 4, fc = X Hz.
- Analyse fréquentielle: FFT, densité spectrale de puissance (Welch).
- Détection d'événements: seuils, corrélation, enveloppe.
- Estimation d'incertitude: répétabilité, résolution ADC, bruit.

Exemples d'analyses à inclure:
- Courbes temporelles (signal brut vs signal traité).
- Spectre amplitude/fréquence.
- Table des paramètres extraits (amplitude, fréquence dominante, rapport signal/bruit).

## Résultats (à remplir)
- Tableau récapitulatif des mesures:
    - Condition | Paramètre mesuré | Valeur | Unité | Incertitude
- Figures:
    - Fig.1: Signal temporel brut
    - Fig.2: Signal après filtrage
    - Fig.3: Spectre (FFT / PSD)

## Discussion
- Comparer résultats attendus vs mesurés.
- Discuter sources d'erreur (bruit, aliasing, dérive thermique).
- Proposer optimisations (augmentation fs, anti-aliasing, blindage, calibration).

## Conclusion
- Résumer acquis et limites.
- Indiquer prochains travaux ou mesures complémentaires.

## Annexes
- Données brutes: chemin/fichier
- Paramètres de configuration DAQ
- Formules utilisées pour conversion/calibration
- Code d'analyse (exemple Python ci-dessous)

## Code d'analyse (exemple Python)
Utiliser ce script pour charger un CSV avec colonnes [time, ch1, ch2...], filtrer et afficher le spectre.